In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras import datasets, layers, models
from keras.layers import GlobalAveragePooling2D, TimeDistributed, Dense, LSTM, GRU, Conv2D, MaxPooling2D, Flatten 
from keras.layers import Input, Rescaling, BatchNormalization, Activation, SeparableConv2D, Reshape, SimpleRNN 
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.inception_v3 import InceptionV3
import os
from pathlib import Path
from imutils import paths
import cv2
from matplotlib import pyplot as plt

In [ ]:
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

In [2]:
input_shape =  (224, 224, 3)
batch_size = 64
time_dist = 8
trsplit = (0.0, 0.7)
valsplit = (0.7, 0.85)
tesplit = (0.85, 1.0)

org_path = Path(r"C:\Users\SWQA\Desktop\STAJ\Metamorphose\Videos\1")
class_paths = [Path(r"class08_30_20"), Path(r"classsenay_1"), Path(r"class16_46_57")]

In [3]:
all_labels = [i for i in os.walk(org_path / class_paths[0] / "Train" / "Label")][0][2]

lie_len = len([i for i in all_labels if i.endswith("l.jpeg")])
err_len = len([i for i in all_labels if i.endswith("e.jpeg")])
both_len = len(all_labels) - lie_len - err_len
none_len = len(list(os.walk(org_path / class_paths[0] / "Train"))[0][2]) - len(all_labels)
all_len = lie_len + err_len + both_len + none_len
print(both_len)
print(lie_len)
print(err_len)
print(none_len)
print(all_len)

class_count = 4

0
7750
216
1769
9735


In [4]:
lie_weight = all_len / ((class_count * lie_len) + 0.000001)
err_weight = all_len / ((class_count * err_len) + 0.000001)
both_weight = all_len / ((class_count * both_len) + 0.000001)
none_weight = all_len / ((class_count * none_len) + 0.000001)
print(both_weight)
print(lie_weight)
print(err_weight)
print(none_weight)

weight_dict = [
    lie_weight,
    err_weight,
    both_weight,
    none_weight
]
weight_dict2 = {
    both_weight: [1, 0],
    lie_weight:  [0, 1],
    err_weight:  [1, 1],
    none_weight: [0, 0]
}

9735000000.0
0.31403225805438606
11.267361098070184
1.3757772751023492


In [149]:
def ins_ind(string, char, index):
        return string[:index] + char + string[index:] 

def __data_gen(wtype, iclass, split):    
    imgs = []    
    lbls = []
    weights = []
    img_dist = []
    lbls = []
    weight_dist = []
    
    bcounter = 0
    
    seed = 0
    for c in wtype:
        seed += ord(c)
    
    np.random.seed(seed)
    shuffled = np.asarray(sorted(list(os.walk(org_path / class_paths[0] / "Train"))[0][2], key=lambda x: len(x)))
    shuffled = shuffled[int(len(shuffled) * split[0]):int(len(shuffled) * split[1])]
    # np.random.shuffle(shuffled)

    #unroll
    for p in shuffled[:time_dist]:        
        img = cv2.imread(str(org_path / iclass / wtype / p), cv2.IMREAD_COLOR)
        img = cv2.resize(img, input_shape[:-1])
        
        img_dist.append(img)

        if os.path.isfile(org_path / iclass / wtype / "Label" / shuffled[time_dist]):
            weight_dist.append(both_weight)
        elif os.path.isfile(org_path / iclass / wtype / "Label" / ins_ind(shuffled[time_dist], 'l', shuffled[time_dist].index('.'))):
            weight_dist.append(lie_weight)
        elif os.path.isfile(org_path / iclass / wtype / "Label" / ins_ind(shuffled[time_dist], 'e', shuffled[time_dist].index('.'))):
            weight_dist.append(err_weight)
        else:
            weight_dist.append(none_weight)

    if os.path.isfile(org_path / iclass / wtype / "Label" / shuffled[time_dist]):
        lbls.append([1, 1])
    elif os.path.isfile(org_path / iclass / wtype / "Label" / ins_ind(shuffled[time_dist], 'l', shuffled[time_dist].index('.'))):
        lbls.append([1, 0])
    elif os.path.isfile(org_path / iclass / wtype / "Label" / ins_ind(shuffled[time_dist], 'e', shuffled[time_dist].index('.'))):
        lbls.append([0, 1])
    else:
        lbls.append([0, 0])


    imgs.append(np.asarray(img_dist, dtype=np.uint8))
    weights.append(np.asarray(weight_dist, dtype=np.float32))

    bcounter += 1

    for p in range(time_dist, len(shuffled) - 1):
        img_index = shuffled[p]
        lbl_index = shuffled[p + 1]

        img = cv2.imread(str(org_path / iclass / wtype / img_index), cv2.IMREAD_COLOR)
        img = cv2.resize(img, input_shape[:-1])

        img_dist.append(img)
        del img_dist[0]

        if os.path.isfile(org_path / iclass / wtype / "Label" / lbl_index):
            lbls.append([1, 1])
            weight_dist.append(both_weight)
        elif os.path.isfile(org_path / iclass / wtype / "Label" / ins_ind(lbl_index, 'l', lbl_index.index('.'))):
            lbls.append([1, 0])
            weight_dist.append(lie_weight)
        elif os.path.isfile(org_path / iclass / wtype / "Label" / ins_ind(lbl_index, 'e', lbl_index.index('.'))):
            lbls.append([0, 1])
            weight_dist.append(err_weight)
        else:
            lbls.append([0, 0])
            weight_dist.append(none_weight)
        del weight_dist[0]

        imgs.append(np.asarray(img_dist, dtype=np.uint8))
        weights.append(np.asarray(weight_dist, dtype=np.float32))

        bcounter += 1

        if bcounter % batch_size == 0:
            bcounter = 0
            yield (np.asarray(imgs, dtype=np.uint8), np.asarray(lbls, dtype=np.float32))
            imgs = []
            lbls = []
            weights = []

def load_data():
    signature = (
        tf.TensorSpec(shape=(batch_size, time_dist, *input_shape), dtype=tf.uint8),
        tf.TensorSpec(shape=(batch_size, *(2,)), dtype=tf.float32),
        # tf.TensorSpec(shape=(batch_size, time_dist), dtype=tf.float32)
    )
    
    for iclass in class_paths:
        trdir_path = org_path / iclass / "Train"
        valdir_path = org_path / iclass / "Val"
        tedir_path = org_path / iclass / "Test"
        
        train_dataset = tf.data.Dataset.from_generator(
            lambda: __data_gen("Train", iclass, trsplit),
            output_signature=signature
        )

        val_dataset = tf.data.Dataset.from_generator(
            lambda: __data_gen("Train", iclass, valsplit),
            output_signature=signature
        )

        test_dataset = tf.data.Dataset.from_generator(
            lambda: __data_gen("Train", iclass, tesplit),
            output_signature=signature
        )
        
        return train_dataset, val_dataset, test_dataset

In [150]:
tr, val, te = load_data()

In [ ]:
tr = tr.shuffle(256)
val = val.shuffle(256)
te = te.shuffle(256)

In [ ]:
for t in tr:
    print(t[1])
    plt.imshow(t[0][0][0], interpolation='nearest')
    plt.show()
    break

In [177]:
sota_cnn = MobileNetV2(include_top=False)

In [178]:
inputs = keras.Input(shape=(time_dist, *input_shape), batch_size=batch_size, dtype=tf.uint8)

In [180]:
def metamorph_loss(weights):
    def inner_loss(y_true, y_pred):
        warr = tf.not_equal(y_true, 1)
        warr = warr

        cw = []
        for w in warr:
            temp = np.empty((1,), dtype=np.float32)
            if tf.reduce_all(tf.math.logical_xor(w, tf.constant([True, False]))):
                temp.fill(weights[0])
            elif tf.reduce_all(tf.math.logical_xor(w, tf.constant([False, True]))):
                temp.fill(weights[1])
            elif tf.reduce_all(tf.math.logical_xor(w, tf.constant([True, True]))):
                temp.fill(weights[2])
            elif tf.reduce_all(tf.math.logical_xor(w, tf.constant([False, False]))):
                temp.fill(weights[3])

            cw.append(temp)
        
        loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0001)(y_true, y_pred, sample_weight=np.asarray(cw))
        
        # tf.print("TRUEEEEEE")
        # tf.print(type(y_true))
        # tf.print(y_true)
        # tf.print("PREDDDDDD")
        # tf.print(type(y_pred))
        # tf.print(y_pred)

        return loss
    
    return inner_loss

In [181]:
xx = TimeDistributed(sota_cnn)(inputs)
xx = TimeDistributed(Flatten())(xx)
xx = LSTM(
    8,
    activation="relu",
    return_sequences=True,
    return_state=True,
    unroll=True,
    stateful=True)(xx)
xx = LSTM(
    8,
    activation="relu",
    return_sequences=True,
    return_state=True,
    unroll=True,
    stateful=True)(xx)
xx = LSTM(
    8,
    activation="relu",
    unroll=True,
    stateful=True)(xx)
out = Dense(2, activation='softmax')(xx)

metamorph = tf.keras.Model(inputs, out)

for layer in sota_cnn.layers:
    layer.trainable = False

metamorph.compile(    
    optimizer='Adam',
    loss=metamorph_loss(weight_dict),
    metrics=[        
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.RootMeanSquaredError(),
        tf.keras.metrics.FalseNegatives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.TruePositives(),
        "binary_accuracy"
    ]
)

In [172]:
metamorph.summary()

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(64, 8, 224, 224,   0           []                               
                                3)]                                                               
                                                                                                  
 time_distributed_36 (TimeDistr  (64, 8, 7, 7, 1280)  2257984    ['input_16[0][0]']               
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (64, 8, 62720)      0           ['time_distributed_36[0][0]']    
 ibuted)                                                                                   

In [182]:
train_len_fe = list(os.walk(org_path / class_paths[0] / "Train"))[0][2]

history1 = metamorph.fit(
    tr.repeat(),
    epochs=100,
    steps_per_epoch=len(train_len_fe[int(len(train_len_fe) * trsplit[0]):int(len(train_len_fe) * trsplit[1])]) // batch_size,
    validation_steps=len(train_len_fe[int(len(train_len_fe) * valsplit[0]):int(len(train_len_fe) * valsplit[1])]) // batch_size,
    validation_data=val.repeat()
)

Epoch 1/100
106/106 [==============================] - 77s 702ms/step - loss: 2846.4741 - mean_squared_error: 0.5649 - root_mean_squared_error: 0.7516 - false_negatives_19: 3673.0000 - true_negatives_19: 3111.0000 - false_positives_19: 4846.0000 - true_positives_19: 1938.0000 - binary_accuracy: 0.3721 - val_loss: 3157.6477 - val_mean_squared_error: 0.5644 - val_root_mean_squared_error: 0.7513 - val_false_negatives_19: 795.0000 - val_true_negatives_19: 613.0000 - val_false_positives_19: 795.0000 - val_true_positives_19: 613.0000 - val_binary_accuracy: 0.4354
Epoch 2/100
106/106 [==============================] - 73s 692ms/step - loss: 2679.7881 - mean_squared_error: 0.5173 - root_mean_squared_error: 0.7192 - false_negatives_19: 2808.0000 - true_negatives_19: 3976.0000 - false_positives_19: 3981.0000 - true_positives_19: 2803.0000 - binary_accuracy: 0.4996 - val_loss: nan - val_mean_squared_error: nan - val_root_mean_squared_error: nan - val_false_negatives_19: 686.0000 - val_true_negati

KeyboardInterrupt: 

In [183]:
dtstnew = te.filter(lambda x, y: tf.reduce_all(y[0] == tf.constant([1., 0.])))

In [73]:
len(list(dtstnew.as_numpy_iterator()))

12

In [175]:
for i in dtstnew.take(1):
    print(i[1])

tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]], shape=(64, 2), dtype=float32)


In [122]:
tf.compat.v1.losses.softmax_cross_entropy(tf.constant([1., 0.]), tf.constant([0.512, 0.65]))

<tf.Tensor: shape=(), dtype=float32, numpy=0.7645258>

In [184]:
metamorph.predict(
    dtstnew.take(1)    
)

1/1 [==============================] - 2s 2s/step


array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan,

In [ ]:
for layer in metamorph.layers[:25]:
   layer.trainable = False
for layer in metamorph.layers[25:]:
   layer.trainable = True
   
from tensorflow.keras.optimizers import SGD
metamorph.compile(
   optimizer=SGD(lr=0.0001, momentum=0.9),
   loss='binary_crossentropy',
   metrics=[
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.RootMeanSquaredError(),
        tf.keras.metrics.F1Score(),
        "accuracy"
   ]
)

In [ ]:
history2 = metamorph.fit(
    tr.repeat(),
    epochs=25,
    batch_size=batch_size,
    steps_per_epoch=len(list(os.walk(org_path / class_paths[0] / "Train"))[0][2]) // batch_size,
    validation_steps=len(list(os.walk(org_path / class_paths[0] / "Val"))[0][2]) // batch_size,
    validation_data=val.repeat()
)

In [ ]:
signature = (tf.TensorSpec(shape=(None, *input_shape), dtype=tf.uint8),
                 tf.TensorSpec(shape=(None, 1), dtype=tf.uint8))

train_dataset2 = tf.data.Dataset.from_generator(
            lambda: __data_gen("Train", class_paths[1]),
            output_signature=signature
        )

In [ ]:
for tt in train_dataset2:
    print(tt[0][1].shape)
    plt.imshow(tt[0][1], interpolation='nearest')
    plt.show()
    break

In [ ]:
metamorph.evaluate(
    te.repeat(),
    steps=len(list(os.walk(org_path / class_paths[0] / "Test"))[0][2]) // batch_size
)

In [ ]:
calc_f1(0.9561, 0.9509)

In [ ]:
W1

false_negatives 0.0
true_negatives  1142.0
false_positives 1.0
true_positives  5033.0

In [ ]:
W0

false_negatives 0.0
true_negatives  1140.0
false_positives 3.0
true_positives  5033.0

In [ ]:
signature = (tf.TensorSpec(shape=(None, *input_shape), dtype=tf.uint8),
                 tf.TensorSpec(shape=(None, 1), dtype=tf.uint8))

train_dataset3 = tf.data.Dataset.from_generator(
            lambda: __data_gen("Train", class_paths[2]),
            output_signature=signature
        )

In [ ]:
metamorph.evaluate(
    train_dataset3.repeat(),
    steps=len(list(os.walk(org_path / class_paths[2] / "Train"))[0][2]) // batch_size
)

In [ ]:
false_negatives 9.0
true_negatives  9058.0
false_positives 105.0
true_positives  204.0